In [1]:
import sys

In [2]:
sys.path

['D:\\Dados\\OneDrive\\Doutorado\\workspace\\bc-playground\\python\\results',
 'D:\\Programas\\Anaconda3\\python37.zip',
 'D:\\Programas\\Anaconda3\\DLLs',
 'D:\\Programas\\Anaconda3\\lib',
 'D:\\Programas\\Anaconda3',
 '',
 'D:\\Programas\\Anaconda3\\lib\\site-packages',
 'D:\\Programas\\Anaconda3\\lib\\site-packages\\win32',
 'D:\\Programas\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'D:\\Programas\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'D:\\Programas\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\thiag\\.ipython']

# Imports

In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

from lib.util.env import getbase_dir
from encrypt import encrypt_data_in_memory
from lib.mybloom.bloomutil import jaccard_coefficient
from splitting_bf.evaluation import split_bf #change method
from splitting_bf.evaluation import simulated_sbf_protocol

import matplotlib.ticker as ticker

#isTrueMatch(gold_standard_dblp_acm,'journals/sigmod/Dogac02',507340)

ModuleNotFoundError: No module named 'lib.util'

# Metodos

## Metodos de simulacao

In [8]:
def split_ds(df,n):
    retorno = []
    for row in df:
        retorno.append([row[0],row[1],split_bf(row[1],n),n])
    return pd.DataFrame(retorno, columns = ['id' , 'bf', 'sbf' , 'n_sbf'])


def calculate_sbf_similarity(dfa,dfb,split_pos=0):
    """
    O split_sim e apenas o do split
    o sbf_sim e de tudo

    :param dfa:
    :param dfb:
    :param split_pos:
    :return:
    """
    result = []
    for indexa, rowa in dfa.iterrows():
        for indexb, rowb in dfb.iterrows():
            full_bf_sim = jaccard_coefficient(rowa[1],rowb[1])
            # print(rowa[2])
            split_sim = jaccard_coefficient(rowa[2][split_pos],rowb[2][split_pos])

            soma = 0
            for i in range(0,len(rowa[2])):
                soma += jaccard_coefficient(rowa[2][i], rowb[2][i])

            result.append({'id_a': rowa[0], 'id_b': rowb[0], 'full_bf': full_bf_sim, 'split_sim': split_sim, 'sbf_sim': soma/len(rowa[2]), 'splits': len(rowa[2])})

    return pd.DataFrame(result)


def private_calculate_precision_recall(zdf,technique):
    clean_r = []
    for index, row in zdf.iterrows():
        # if count % 100 == 0:
        #     print(count)
        # count += 1

        # id_max = zdf[(zdf['id_a'] == row['id_a']) & (zdf['id_b'] == row['id_b'])]['sbf_sim'].idxmax()
        id_max_a = zdf[(zdf['id_a'] == row['id_a'])][technique].idxmax()
        id_max_b = zdf[(zdf['id_b'] == row['id_b'])][technique].idxmax()

        if (id_max_a == id_max_b) and (id_max_a == index):
            # print(row)
            rp = {'id_a': row['id_a'], 'id_b': row['id_b'], 'sbf_sim': row['sbf_sim'], 'bf_sim': row['bf_sim'],
                  'sbf_stat': row['sbf_stat'],'bf_stat': row['bf_stat']}
            clean_r.append(rp)

    return pd.DataFrame(clean_r)


    zdf = x_final
    gs = gs_dict
    for a in zdf.alfa.unique():
        max_p_bf = -1
        max_r_bf = -1
        for e in zdf.beta_error.unique():
            for s in zdf.sbf_splits.unique():
                # quociente full bf
                clean_bf = zdf[(zdf.alfa == a) & (zdf.beta_error == e) & (zdf.sbf_splits == s)]
                qp_bf = len(clean_bf[clean_bf.bf_stat == 'FM']) + len(clean_bf[clean_bf.bf_stat == 'TM'])
                if qp_bf != 0:
                    p_bf = len(clean_bf[clean_bf.bf_stat == 'TM']) / qp_bf
                else:
                    print('aqui')
                    p_bf = 0

                r_bf = len(clean_bf[clean_bf.bf_stat == 'TM']) / len(gs)
                max_p_bf = max(max_p_bf,p_bf)
                max_r_bf = max(max_r_bf,r_bf)

## Metodos de plotagem

In [1]:
def plot_line_metric(dfm, title, ds='dblp_acm', dir='sbf_03'):
    fig, ax = plt.subplots(figsize=(8, 5))
    sns.lineplot(data=pf, x='threshold_alfa', y='vals', hue='bf_type',
                 linewidth=2.5, dashes=[(0, 0), (2, 2)])
    # sns.barplot(data=pf, x='threshold_alfa', y='vals', hue='cols',
    #             linewidth=2.5)
    # palette = "tab10"
    ax.set_title(title)
    ax.set_ylabel("")
    ax.grid('on')
    plt.tight_layout()
    plt.show()
    fig.savefig(file)
    file = getbase_dir(['results', dir, ds + title + '.png'])


def plot_metric(df,metric='f1'):
    """

    :param df:
    :param metric: recall, precision and f1 (default)
    :return:
    """

    # global fig, ax, y, x
    fig, ax = plt.subplots(figsize=(8, 5))
    sns.barplot(data=df, x='alfa', y=metric+'_sbf',
                capsize=.2, color='gray')
    # ax2 = ax.twinx()
    y = []
    for x in df.alfa.unique():
        y.append(df[df.alfa == x][metric+'_bf_full'].max())
    ax.plot(ax.get_xticks(), y, color='red',
            linestyle='--', linewidth=3, label='full_bf')
    ax.grid('on')
    # ax2.grid(False)
    # plt.tight_layout()
    plt.show()
    plt.close()
    plt.clf()

def plot_all_metrics(df,ds='dblp-acm'):
    """

    :param df:
    :param metric: recall, precision and f1 (default)
    :return:
    """

    # global fig, ax, y, x
    fig, axs = plt.subplots(figsize=(12, 5), ncols=3)
    metrics = ['precision', 'recall', 'f1']
    for i in range(0, len(axs)):
        ax = axs[i]
        metric = metrics[i]
        sns.barplot(data=df, x='alfa', y=metrics[i] + '_sbf',
                    capsize=.1, color='skyblue', alpha=0.8 ,
                    ax=ax)
        # sns.barplot(data=df, x='alfa', y=metrics[i] + '_sbf',
        #             capsize=.2, ax=ax)

        ax.set_title(metric)

        y = []
        for x in df.alfa.unique():
            y.append(df[df.alfa == x][metrics[i] + '_bf_full'].mean())

        ax.plot(ax.get_xticks(), y, color='red',
                linestyle='--', linewidth=2, label='full_bf')
        ax.grid('on')
        ax.set_ylabel("")
        ax.set_xlabel(r'$\alpha$')

    plt.tight_layout()
    plt.show()
    plt.close()
    plt.clf()
    file = getbase_dir(['results', 'abel_01']) + 'plot_all_metrics_considering_alfas_' + ds + '.png'
    fig.savefig(file, dpi=600)

# Processamento

## variaveis de ambiente

In [9]:
gold_standard_dblp_acm = pd.read_csv(getbase_dir(['Datasets', 'leipzig_dblp_acm']) + 'DBLP-ACM_perfectMapping.csv',
                                         sep=',')
gs_dict = {}
for indexa, row in gold_standard_dblp_acm.iterrows():
    gs_dict[str(row[0])+str(row[1])] = True

del gold_standard_dblp_acm

## Simulação

### Calcular similaridade

##### Carregar variaveis

In [ ]:
acm = pd.read_csv(getbase_dir(['Datasets', 'leipzig_dblp_acm']) + 'ACM.csv', sep=',')
dblp = pd.read_csv(getbase_dir(['Datasets', 'leipzig_dblp_acm']) + 'DBLP2.csv', sep=',', encoding='latin')

eacm = encrypt_data_in_memory(acm, [1, 2, 3, 4], 1024)
edblp = encrypt_data_in_memory(dblp, [1, 2, 3, 4], 1024)

### Computar similaridades

In [ ]:
    splits_lenght = [3 , 4 , 5 , 6 , 7 , 8]
    similarity = []

    for s in splits_lenght:
        print(s)
        #df_acm = split_ds(eacm[0:10], s)
        #df_dblp = split_ds(edblp[0:10], s)
        df_acm = split_ds(eacm, s)
        df_dblp = split_ds(edblp, s)
        import time
        start_time = time.time()
        z = calculate_sbf_similarity(df_dblp,df_acm)
        print("\t %s seconds" % (time.time() - start_time))
        if len(similarity) == 0:
            similarity = z
        else:
            similarity = pd.concat([similarity, z])

    del df_acm
    del df_dblp

    similarity.to_csv(getbase_dir(['results', 'abel_01']) + 'dblp_acm_sim.csv')

### Simular protocolo

#### Caregar similaridade

In [10]:
similarity = pd.read_csv(getbase_dir(['results', 'abel_01']) + 'dblp_acm_sim.csv')
# z = similarity[(similarity.id_a=='conf/vldb/PoosalaI96') & (similarity.id_b==304587)]
print(similarity.memory_usage(index=True).sum()/1024/1024)

1921.3882446289062


ajuste nos dados se necessario

In [11]:
# len(similarity)
    # len(similarity[similarity.splits.isin([3,5,8])])
len(similarity)
s = similarity[similarity['splits'].isin([8, 16,32,64,128])]
len(s)
del similarity
similarity = s
del s
del indexa
del row

In [12]:
del acm
del dblp
del eacm
del edblp

#### Simulação do protocolo

In [13]:
chunks = np.array_split(similarity, 6)
del similarity

In [ ]:
# alfas_t = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
#v = 'alfa1'
v = 'alfa5e'
alfas_t = [.5]
x_final = []
errors_a5 = [.001,.01,.025,.5,.15,.2,.25,.3,.37]
# from multiprocessing.pool import ThreadPool
# pool = ThreadPool(processes=8)
pool = mp.Pool(processes=4)

for c in range(0,len(chunks)):
    sim_chunk = chunks.pop(0)
    
    print("=========== chunk: " + str(c))
    
    total = 0
    for k in range(0,len(chunks)):
        total += sim_chunk.memory_usage(index=True).sum()/1024/1024
    print(total)
    
    for alfa in alfas_t:
        print("=== " + str(alfa))
        # x.to_csv('x.csv',index=False)
        # x = pd.read_csv('x.csv')

        # stand alone
        # x = simulated_sbf_protocol(similarity, gs_dict, alfa , error= 0.1)
        # para utilizar  a simulaçao

        print("\t Poll criado")

        # results = [pool.imap(simulated_sbf_protocol,
        results = [pool.apply_async(simulated_sbf_protocol,
                                    args=(sim_chunk, gs_dict, alfa),
                                    # args=(similarity, gs_dict, alfa),
#                                     kwds={'error': e}) for e in np.arange(0.01, .2, 0.04)
                                    kwds={'error': e}) for e in errors_a5
                                    # kwds={'error': e}) for e in np.arange(0.01, .4, 0.9)
                                    #kwds={'error': e}) for e in np.arange(0.01, .4, 0.07)
                   ]

        output = [p.get() for p in results]

        x = output[0]
        for pdf in output[1:]:
            x = x.append(pdf, ignore_index=True)

        print("\t Resultados capturados")
        # salva os resultados intermediarops
#         x.to_csv(
#             getbase_dir(['results', 'abel_01']) + 'intermediate_dblp_acm_'+v+'_chunk' + str(c) + '_' + str(alfa) + '.csv')
#         print("\t Resultados salvos")
    del sim_chunk
    
pool.close()
#         if len(x_final) == 0:
#             x_final = x
#         else:
#             x_final = pd.concat([x_final, x])

del chunks, output
del pool
del pd
del results
#### fora do for
x_final.to_csv(getbase_dir(['results', 'abel_01']) + 'dblp_acm_x_final_'+v+'.csv')

=========== chunk: 0
1524.9114990234375
=== 0.5
	 Poll criado


In [ ]:
# recuperar pelos intermediarios
x_final = []
for alfa in alfas_t:
    for c in range(1,13): 
        x = getbase_dir(['results', 'abel_01']) + 'intermediate_dblp_acm_'+v+'_chunk' + str(c) + '_' + str(alfa) + '.csv')
        if len(x_final) == 0:
            x_final = x
        else:
            x_final = pd.concat([x_final, x])

x_final.to_csv(getbase_dir(['results', 'abel_01']) + 'dblp_acm_x_final_'+v+'.csv')

#### Salvandos os resultados

In [ ]:
def calculate_precision_recall(zdf, gs, ds_name , full_bf_len=1024):

    # zdf = fdf[fdf.abel_1st_s == True]
    count = 0

    saida = []
    for a in zdf.alfa.unique():
        max_p_bf = -1
        max_r_bf = -1
        for e in zdf.beta_error.unique():
            for s in zdf.sbf_splits.unique():
                # quociente full bf'
                clean_bf = zdf[(zdf.alfa == a) & (zdf.beta_error == e) & (zdf.sbf_splits == s)]
                qp_bf = len(clean_bf[clean_bf.bf_stat == 'FM']) + len(clean_bf[clean_bf.bf_stat == 'TM'])
                if qp_bf != 0:
                    p_bf = len(clean_bf[clean_bf.bf_stat == 'TM']) / qp_bf
                else:
                    print('aqui ')
                    p_bf = 0

                r_bf = len(clean_bf[clean_bf.bf_stat == 'TM']) / len(gs)
                max_p_bf = max(max_p_bf, p_bf)
                max_r_bf = max(max_r_bf, r_bf)


        for e in zdf.beta_error.unique():
            for s in zdf.sbf_splits.unique():
                #Colocar true no 1st_la
                tdf = zdf[(zdf.abel_1st_s == True ) & (zdf.alfa == a) & (zdf.beta_error == e) & (zdf.sbf_splits == s)]
                clean_sbf = tdf

                # cociente precision
                qp_spf = len(clean_sbf[clean_sbf.sbf_stat == 'FM']) + len(clean_sbf[clean_sbf.sbf_stat == 'TM'])
                if qp_spf != 0:
                    p_sbf = len(clean_sbf[clean_sbf.sbf_stat == 'TM']) / qp_spf
                else:
                    p_sbf = 0

                r_sbf = len(clean_sbf[clean_sbf.sbf_stat == 'TM']) / len(gs)

                saida.append([ds_name, s, full_bf_len, a, e, p_sbf, r_sbf, max_p_bf, max_r_bf])

    return pd.DataFrame(saida, columns = ['ds_name', 'sbf_splits' , 'bf_len' ,
                                          'alfa' , 'beta_error', 'precision_sbf' ,
                                          'recall_sbf' , 'precision_bf_full' , 'recall_bf_full' ])

salvando os resultados finais

In [ ]:
final_result = calculate_precision_recall(x_final, gs_dict, 'dblp_acm')
final_result['f1_bf_full'] = 2 * ((final_result['precision_bf_full'] * final_result['recall_bf_full']) / (final_result['precision_bf_full'] + final_result['recall_bf_full']))
final_result['f1_sbf'] = 2 * ((final_result['precision_sbf'] * final_result['recall_sbf']) / (final_result['precision_sbf'] + final_result['recall_sbf']))
final_result['ls'] = (final_result.bf_len / final_result.sbf_splits) / final_result.bf_len
final_result.to_csv(getbase_dir(['results', 'abel_01']) + 'dblp_acm_final_result_'+v+'.csv')

## Graficos

### carregandos os dados

In [ ]:
#v = 'b1' #old computado errado
v = 'alfa5'

rf = 'dblp_acm_final_result_'+v+'.csv'
xf = 'dblp_acm_x_final_'+v+'.csv'

x_final = pd.read_csv(getbase_dir(['results', 'abel_01']) + xf)
final_result = pd.read_csv(getbase_dir(['results', 'abel_01']) + rf)


In [ ]:
def calculate_precision_recall(zdf, gs, ds_name , full_bf_len=1024):

    # zdf = fdf[fdf.abel_1st_s == True]
    count = 0

    saida = []
    for a in zdf.alfa.unique():
        max_p_bf = -1
        max_r_bf = -1
        for e in zdf.beta_error.unique():
            for s in zdf.sbf_splits.unique():
                # quociente full bf'
                clean_bf = zdf[(zdf.alfa == a) & (zdf.beta_error == e) & (zdf.sbf_splits == s)]
                
                TP = len(clean_bf[clean_bf.bf_stat == 'TM'])
                #FALSE POSITIVE
                FP = len(clean_bf[clean_bf.bf_stat == 'FM'])
                #FALSE NEGATIVE
                FN = len(gs) - TP
                
                qp_spf = TP + FP
                if qp_spf != 0:
                    p_bf = TP / qp_spf
                else:
                    p_sbf = 0
                
                r_bf = TP / (TP + FN)
                
                max_p_bf = max(max_p_bf, p_bf)
                max_r_bf = max(max_r_bf, r_bf)
#                 print(max_r_bf)
        
        del clean_bf
        del p_bf
        del r_bf


        for e in zdf.beta_error.unique():
            for s in zdf.sbf_splits.unique(): 
                
                #Colocar true no 1st_la
                clean_sbf = zdf[(zdf.abel_1st_s == True ) & (zdf.alfa == a) & (zdf.beta_error == e) & (zdf.sbf_splits == s)]
                
                #TRUE POSITIVE
                TP = len(clean_sbf[clean_sbf.sbf_stat == 'TM'])
                #FALSE POSITIVE
                FP = len(clean_sbf[clean_sbf.sbf_stat == 'FM'])
                #FALSE NEGATIVE
                FN = len(gs) - TP               
                
                # cociente precision
                qp_spf = TP + FP
                if qp_spf != 0:
                    p_sbf = TP / qp_spf
                else:
                    p_sbf = 0

                r_sbf = TP / (TP + FN)

                saida.append([ds_name, s, full_bf_len, a, e, p_sbf, r_sbf, max_p_bf, max_r_bf])

    return pd.DataFrame(saida, columns = ['ds_name', 'sbf_splits' , 'bf_len' ,
                                          'alfa' , 'beta_error', 'precision_sbf' ,
                                          'recall_sbf' , 'precision_bf_full' , 'recall_bf_full' ])



In [ ]:
final_result = calculate_precision_recall(x_final,gs_dict,'dblp-acm')
final_result['f1_bf_full'] = 2 * ((final_result['precision_bf_full'] * final_result['recall_bf_full']) / (final_result['precision_bf_full'] + final_result['recall_bf_full']))
final_result['f1_sbf'] = 2 * ((final_result['precision_sbf'] * final_result['recall_sbf']) / (final_result['precision_sbf'] + final_result['recall_sbf']))
final_result['ls'] = (final_result.bf_len / final_result.sbf_splits) / final_result.bf_len
final_result.to_csv(getbase_dir(['results', 'abel_01']) + 'dblp_acm_final_result_alfa5.csv')

ajustes nos dados 

In [ ]:
final_result = final_result[final_result.sbf_splits != max(final_result.sbf_splits.unique())]

### Figura 1

Mericas individualizadas

In [ ]:
plot_metric(final_result, metric='f1')
plot_metric(final_result,metric='precision')
plot_metric(final_result, metric='recall')

Todas as metricas

In [ ]:
plot_all_metrics(final_result)

teste estatiscos da figura 1

In [ ]:
from scipy.stats import wilcoxon , ttest_ind
#  null hypothesis of identical average scores. (< 0.05)
for metric in ['f1','precision','recall']:
    y_bf = []
    y_sbf = []
    # metric = 'f1'
    for x in final_result.alfa.unique():
        y_bf.append(final_result[final_result.alfa == x][metric + '_bf_full'].mean())
        y_sbf.append(final_result[final_result.alfa == x][metric + '_sbf'].median())
    print(metric)
        # print(wilcoxon(final_result[metric + '_bf_full'], final_result[metric + '_bf_full']))
    print(ttest_ind(y_bf,y_sbf))

### Figura 2

#### Metodo da plotagem

In [ ]:
def plot_quality_considering_error(df,limiar,metric='f1'):

    ldf = df[df.alfa == limiar].round(3)
    ldf.ls = ldf.ls.astype(str) + '%'
    # ldf['ls'] = (final_result.bf_len / final_result.sbf_splits) / final_result.bf_len
    # ldf.sbf_splits = ldf.sbf_splits.astype(str)
    fig, ax = plt.subplots(figsize=(8, 5))
    # palette = dict(zip(ldf.sbf_splits.unique(), sns.color_palette()))
    # palette.update({"Total": "k"})
    sns.lineplot(data=ldf, x='beta_error', y=metric+'_sbf', hue="ls")
    y = []
    for x in ax.get_xticks():
        y.append(df[metric+'_bf_full'].max())
    ax.plot(ax.get_xticks(), y, color='red',
            linestyle='--', linewidth=3, label='Sine')
    # sns.lineplot(data=ldf, x='beta_error', y='f1_sbf', hue="sbf_splits", palette=palette)
    ax.grid(True)
    plt.show()
    plt.close()
    plt.clf()

def plot_all_quality_considering_error(df,limiar,ds='dblp-acm',pontos=[]):
    """

    :param df:
    :param limiar:
    :param ds:
    :return:
    """

    ldf = df[df.alfa == limiar]

    ldf['z'] = df.bf_len / df.sbf_splits
    ldf['split_length'] = ldf.ls.round(3).astype(str) + '% '
    ldf.split_length = ldf.split_length + '(' +ldf.z.astype(str) + ' bits)'
    ldf['beta'] = limiar - ldf.beta_error
    # min_x = ldf.beta.min()
    # max_x = ldf.beta.max()
    min_x = ldf.beta_error.min()
    max_x = ldf.beta_error.max()

    fig, axs = plt.subplots(figsize=(12, 5), ncols=3)
    metrics = ['precision', 'recall', 'f1']
    for i in range(0, len(axs)):
        metric = metrics[i]

        ax = axs[i]
        ax.set_xlim(min_x,max_x)
        # ax.set_xlim(max_x,min_x)
        ax.set_title(r"" +metric + " $\\alpha$ ="+ str(limiar) )

        g = sns.lineplot(data=ldf, x='beta_error', y=metric + '_sbf',
                         hue="split_length", ax=ax)
        # g = sns.lineplot(data=ldf, x='beta', y=metric+'_sbf',
        #                 hue="split_length", ax=ax)

        ax.set_ylabel("")
        ax.set_xlabel(r'error ($\beta = \alpha - error)$')

        y = []
        for x in ax.get_xticks():
            y.append(ldf[metric+'_bf_full'].mean())

        ax.plot(ax.get_xticks(), y, color='red',
                linestyle='--', linewidth=2, label='full_bf')

        #### errros
        if len(pontos) != 0:
            if metric != 'f1-':
                cores = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
                cc = 0
                for vx in pontos.round(2).y:
                    # ax.axvline(x=vx, linewidth=2, c=c.colors[cc], alpha=0.8)
                    ax.axvline(x=vx, linewidth=10, c=cores[cc], alpha=0.2)
                    cc+=1
            # ax.scatter(pontos.x, pontos.y ,c=pontos.x, #s=scale, label=color,
            #         alpha=0.8, edgecolors='none')

            # ax.axhline(pontos.y, linewidth=2, c=pontos.x, alpha=0.8)


        ax.grid(True)
        if i == 1:
            g.legend(loc='lower center', bbox_to_anchor= (0.5, -.45) ,
                     borderaxespad=0, frameon=False,
                     fontsize = 'medium', ncol=2 )
        else:
            ax.get_legend().remove()
            # fontsize: int or float or {'xx-small', 'x-small', 'small', 'medium', 'large', 'x-large', 'xx-large'}
    plt.tight_layout()
    plt.show()
    plt.close()
    plt.clf()
    if len(pontos) == 0:
        file = getbase_dir(['results', 'abel_01']) + 'plot_all_quality_considering_error_'+ds+'_alfa_'+str(limiar)+'.png'
    else:
        file = getbase_dir(['results', 'abel_01']) + 'plot_all_quality_considering_error_' +ds+ '_alfa_' + str(limiar) + '_width_region.png'
    fig.savefig(file,dpi=800)


#### Plotagem

Definindo as regiões beta

In [ ]:
a = -0.042876301194393125
b = 3.2574724870013103

fe1 = lambda x, a, b: a * np.log(b * x)
mi = fe1(.99999,a,b)
ma = fe1(.00001,a,b)
#fe1 = lambda x: -np.log(x)
#mi = fe1(.9999999999999)
#ma = fe1(.0000000000001)

normalize = lambda x,min,max:  (x - min) / (max - min)

pontos = []
for i in final_result.ls.unique():
    # print("{} & {} & \\\\".format(i,fe1(i,a,b)))
    #n = normalize(fe1(i), mi, ma)
    x = fe1(i,a,b)
    n = normalize(x, mi, ma)
    
    pontos.append([i,n,x])
    #print("{0:.3f} & {1:.2f} \\\\".format(i, n))

pontos = pd.DataFrame(pontos,columns=['x','y','z'])
pontos

#### Resultado

In [ ]:
#plot_all_quality_considering_error(final_result, .5 ,pontos=pontos[:-1])
#ldf = final_result[final_result.alfa==.5]
#plot_all_quality_considering_error(ldf, .5 ,pontos=pontos)
plot_all_quality_considering_error(final_result, .5 ,pontos=pontos)

plota o restante dos graficos

In [ ]:
plot_all_quality_considering_error(final_result,.4)
plot_all_quality_considering_error(final_result, .5)
plot_all_quality_considering_error(final_result,.6)
plot_all_quality_considering_error(final_result,.7)
plot_all_quality_considering_error(final_result,.8)
plot_all_quality_considering_error(final_result,.9)

In [ ]:
limiar = .5
df =  x_final[(x_final.alfa == limiar)]
df['beta'] = limiar - df.beta_error
x = df[df.abel_1st_s == True].round(3).beta.astype(str)
df

In [ ]:
n_ds1=2617
n_ds2=2294

t = n_ds1 * n_ds2
df['x'] = df[df.abel_1st_s == True].round(3).beta_error.astype(str)
df['y'] = 1 / t
df['y'] = 1
from matplotlib.ticker import PercentFormatter


total = len(df[df.x == '0.33'])

fig, ax = plt.subplots(figsize=(8, 5))
# g = sns.catplot(x="x", y="y", hue="y", kind="bar", data=df)
# for ax in g.axes.flat:
#     ax.yaxis.set_major_formatter(PercentFormatter(1))
# plt.show()
# g = sns.catplot(x="x", kind="count", data=df)

g = sns.countplot(x="x", data=df)  # for Seaborn version 0.7 and more
for p in g.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2.,
            height + 3,
            '{:1.2f}'.format(height),
            ha="center")
ax.grid(True)
plt.show()
plt.close()
plt.clf()

#file = getbase_dir(['results', 'abel_01']) + 'zzz_05_alfa_' + '.png'
#fig.savefig(file, dpi=400)

# Scratch

In [ ]:
    # final_result[(final_result.sbf_splits != 128) & (final_result.sbf_splits != 64)

    n_ds1=2617
    n_ds2=2294

    t = n_ds1 * n_ds2
    df['x'] = df[df.abel_1st_s == True].round(3).beta.astype(str)
    df['y'] = 1 / t
    df['y'] = 1
    from matplotlib.ticker import PercentFormatter


    total = len(df[df.x == '0.33'])

    fig, ax = plt.subplots(figsize=(8, 5))
    # g = sns.catplot(x="x", y="y", hue="y", kind="bar", data=df)
    # for ax in g.axes.flat:
    #     ax.yaxis.set_major_formatter(PercentFormatter(1))
    # plt.show()
    # g = sns.catplot(x="x", kind="count", data=df)

    g = sns.countplot(x="x", data=df)  # for Seaborn version 0.7 and more
    for p in g.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width() / 2.,
                height + 3,
                '{:1.2f}'.format(height / total),
                ha="center")
    ax.grid(True)
    plt.show()
    plt.close()
    plt.clf()

    file = getbase_dir(['results', 'abel_01']) + 'zzz_05_alfa_' + '.png'
    fig.savefig(file, dpi=400)



##########################################################################
### scratch
########################################################################
def exec_01(n):
    import time
    start_time = time.time()
    ids_a = [640999,190649, 615205, 375780, 375736]
    ids_b = ['journals/sigmod/Winslett03','journals/sigmod/Kim94', 'conf/sigmod/BlakeleyD00', 'conf/sigmod/NazeriBO01', 'conf/sigmod/BorkarDS01' ]
    for a in ids_a:
        for b in ids_b:
            print(isTrueMatchOriginal(gs, b, a))
    print(time.time() - start_time)


print(exec_01(1))

gs =  gold_standard_dblp_acm.copy()

gs.set_index(['idDBLP', 'ix'])

ida = 'journals/sigmod/Winslett03'
idb = 640999


d = {}
for indexa, row in gs.iterrows():
    d[str(row[0])+str(row[1])] = True

def isTrueMatch2(gs,ida,idb,base_a='idDBLP',base_b='idACM'):
    try:
        gs[str(ida)+str(idb)]
        return True
    except KeyError:
        print
        return False
    #     # gs[(gs[base_a] == ida) & (gs[base_b] == idb)]
    # if len(z) == 1:
    #     return True
    #
    # return False